In [1]:
import pandas as pd
import numpy as np

from metrics import NNSE,ZNRMSE,PPTS,LPTS



Evaluate the calibration and validation performance of different models


In [2]:
hydro_stations = ['Tangnaihai','Guide','Xunhua']
station_channel = {
    'Tangnaihai':3,
    'Guide':13,
    'Xunhua':9
}
metrics_result = pd.DataFrame(
    index=hydro_stations,
    columns=['CAL_NNSE_SWATPlus','CAL_NNSE_ARIMA_Additive','CAL_NNSE_ARIMA_Multiplicative',
             'CAL_ZNRMSE_SWATPlus','CAL_ZNRMSE_ARIMA_Additive','CAL_ZNRMSE_ARIMA_Multiplicative',
             'CAL_PPTS_SWATPlus','CAL_PPTS_ARIMA_Additive','CAL_PPTS_ARIMA_Multiplicative',
             'CAL_LPTS_SWATPlus','CAL_LPTS_ARIMA_Additive','CAL_LPTS_ARIMA_Multiplicative',
             'VAL_NNSE_SWATPlus','VAL_NNSE_ARIMA_Additive','VAL_NNSE_ARIMA_Multiplicative',
             'VAL_ZNRMSE_SWATPlus','VAL_ZNRMSE_ARIMA_Additive','VAL_ZNRMSE_ARIMA_Multiplicative',
             'VAL_PPTS_SWATPlus','VAL_PPTS_ARIMA_Additive','VAL_PPTS_ARIMA_Multiplicative',
             'VAL_LPTS_SWATPlus','VAL_LPTS_ARIMA_Additive','VAL_LPTS_ARIMA_Multiplicative']
)
metrics_result.index.name = 'station'
for hydro_station in hydro_stations:
    obs = pd.read_csv(
        f'../data/{hydro_station.lower()}_natural_monthly_flow.csv',
        parse_dates=['date'],
        index_col=['date']
    )
    obs_cal = obs['1972-01-01':'2008-12-31']
    obs_val = obs['2009-01-01':'2014-12-31']
    
    swatplus_calval = pd.read_csv(
        f'../result/SWATPlusCalValSimData/Channel_{station_channel[hydro_station]}_Monthly_River-Flow_{hydro_station}_Sim1972_2019.csv',
        parse_dates=['Date'],
        index_col=['Date']
    )
    swatplus_cal = swatplus_calval['1972-01-01':'2008-12-31']
    swatplus_val = swatplus_calval['2009-01-01':'2014-12-31']

    arima_cal_additive = pd.read_csv(
        f'../result/ARIMAPredData/seasonal_decompose_additive_arima_train_sim_{hydro_station.lower()}_before_2009.csv',
        parse_dates=['date'],
        index_col=['date']
    )

    arima_cal_additive = arima_cal_additive['1972-01-01':'2008-12-31']

    arima_val_additive = pd.read_csv(
        f'../result/ARIMAPredData/seasonal_decompose_additive_arima_pred_{hydro_station.lower()}_2009_2014.csv',
        parse_dates=['date'],
        index_col=['date']
    )

    arima_cal_multiplicative = pd.read_csv(
        f'../result/ARIMAPredData/seasonal_decompose_multiplicative_arima_train_sim_{hydro_station.lower()}_before_2009.csv',
        parse_dates=['date'],
        index_col=['date']
    )

    arima_cal_multiplicative = arima_cal_multiplicative['1972-01-01':'2008-12-31']

    arima_val_multiplicative = pd.read_csv(
        f'../result/ARIMAPredData/seasonal_decompose_multiplicative_arima_pred_{hydro_station.lower()}_2009_2014.csv',
        parse_dates=['date'],
        index_col=['date']
    )

    df_new_cal = pd.DataFrame(obs_cal.index)
    df_new_cal.index.name = 'date'
    df_new_cal['obs'] = obs_cal['flow(m^3/s)'].values
    df_new_cal['swatplus'] = swatplus_cal['Value'].values
    df_new_cal['arima_additive'] = arima_cal_additive['SimFlow(m^3/s)'].values
    df_new_cal['arima_multiplicative'] = arima_cal_multiplicative['SimFlow(m^3/s)'].values

    cal_nnse_swatplus = NNSE(df_new_cal['obs'].values, df_new_cal['swatplus'].values)
    cal_nnse_arima_additive = NNSE(df_new_cal['obs'].values, df_new_cal['arima_additive'].values)
    cal_nnse_arima_multiplicative = NNSE(df_new_cal['obs'].values, df_new_cal['arima_multiplicative'].values)

    cal_znrmse_swatplus = ZNRMSE(df_new_cal['obs'].values, df_new_cal['swatplus'].values)
    cal_znrmse_arima_additive = ZNRMSE(df_new_cal['obs'].values, df_new_cal['arima_additive'].values)
    cal_znrmse_arima_multiplicative = ZNRMSE(df_new_cal['obs'].values, df_new_cal['arima_multiplicative'].values)

    cal_ppts_swatplus = PPTS(df_new_cal['obs'].values, df_new_cal['swatplus'].values)
    cal_ppts_arima_additive = PPTS(df_new_cal['obs'].values, df_new_cal['arima_additive'].values)
    cal_ppts_arima_multiplicative = PPTS(df_new_cal['obs'].values, df_new_cal['arima_multiplicative'].values)

    cal_lpts_swatplus = LPTS(df_new_cal['obs'].values, df_new_cal['swatplus'].values)   
    cal_lpts_arima_additive = LPTS(df_new_cal['obs'].values, df_new_cal['arima_additive'].values)
    cal_lpts_arima_multiplicative = LPTS(df_new_cal['obs'].values, df_new_cal['arima_multiplicative'].values)

    df_new_val = pd.DataFrame(obs_val.index)
    df_new_val.index.name = 'date'
    df_new_val['obs'] = obs_val['flow(m^3/s)'].values
    df_new_val['swatplus'] = swatplus_val['Value'].values
    df_new_val['arima_additive'] = arima_val_additive['flow(m^3/s)'].values
    df_new_val['arima_multiplicative'] = arima_val_multiplicative['flow(m^3/s)'].values

    val_nnse_swatplus = NNSE(df_new_val['obs'].values, df_new_val['swatplus'].values)
    val_nnse_arima_additive = NNSE(df_new_val['obs'].values, df_new_val['arima_additive'].values)
    val_nnse_arima_multiplicative = NNSE(df_new_val['obs'].values, df_new_val['arima_multiplicative'].values)

    val_znrmse_swatplus = ZNRMSE(df_new_val['obs'].values, df_new_val['swatplus'].values)
    val_znrmse_arima_additive = ZNRMSE(df_new_val['obs'].values, df_new_val['arima_additive'].values)
    val_znrmse_arima_multiplicative = ZNRMSE(df_new_val['obs'].values, df_new_val['arima_multiplicative'].values)

    val_ppts_swatplus = PPTS(df_new_val['obs'].values, df_new_val['swatplus'].values)
    val_ppts_arima_additive = PPTS(df_new_val['obs'].values, df_new_val['arima_additive'].values)
    val_ppts_arima_multiplicative = PPTS(df_new_val['obs'].values, df_new_val['arima_multiplicative'].values)

    val_lpts_swatplus = LPTS(df_new_val['obs'].values, df_new_val['swatplus'].values)
    val_lpts_arima_additive = LPTS(df_new_val['obs'].values, df_new_val['arima_additive'].values)
    val_lpts_arima_multiplicative = LPTS(df_new_val['obs'].values, df_new_val['arima_multiplicative'].values)

    metrics_result.loc[hydro_station, 'CAL_NNSE_SWATPlus'] = cal_nnse_swatplus
    metrics_result.loc[hydro_station, 'CAL_NNSE_ARIMA_Additive'] = cal_nnse_arima_additive
    metrics_result.loc[hydro_station, 'CAL_NNSE_ARIMA_Multiplicative'] = cal_nnse_arima_multiplicative

    metrics_result.loc[hydro_station, 'CAL_ZNRMSE_SWATPlus'] = cal_znrmse_swatplus
    metrics_result.loc[hydro_station, 'CAL_ZNRMSE_ARIMA_Additive'] = cal_znrmse_arima_additive
    metrics_result.loc[hydro_station, 'CAL_ZNRMSE_ARIMA_Multiplicative'] = cal_znrmse_arima_multiplicative

    metrics_result.loc[hydro_station, 'CAL_PPTS_SWATPlus'] = cal_ppts_swatplus
    metrics_result.loc[hydro_station, 'CAL_PPTS_ARIMA_Additive'] = cal_ppts_arima_additive
    metrics_result.loc[hydro_station, 'CAL_PPTS_ARIMA_Multiplicative'] = cal_ppts_arima_multiplicative

    metrics_result.loc[hydro_station, 'CAL_LPTS_SWATPlus'] = cal_lpts_swatplus
    metrics_result.loc[hydro_station, 'CAL_LPTS_ARIMA_Additive'] = cal_lpts_arima_additive
    metrics_result.loc[hydro_station, 'CAL_LPTS_ARIMA_Multiplicative'] = cal_lpts_arima_multiplicative

    metrics_result.loc[hydro_station, 'VAL_NNSE_SWATPlus'] = val_nnse_swatplus
    metrics_result.loc[hydro_station, 'VAL_NNSE_ARIMA_Additive'] = val_nnse_arima_additive
    metrics_result.loc[hydro_station, 'VAL_NNSE_ARIMA_Multiplicative'] = val_nnse_arima_multiplicative

    metrics_result.loc[hydro_station, 'VAL_ZNRMSE_SWATPlus'] = val_znrmse_swatplus
    metrics_result.loc[hydro_station, 'VAL_ZNRMSE_ARIMA_Additive'] = val_znrmse_arima_additive
    metrics_result.loc[hydro_station, 'VAL_ZNRMSE_ARIMA_Multiplicative'] = val_znrmse_arima_multiplicative

    metrics_result.loc[hydro_station, 'VAL_PPTS_SWATPlus'] = val_ppts_swatplus
    metrics_result.loc[hydro_station, 'VAL_PPTS_ARIMA_Additive'] = val_ppts_arima_additive
    metrics_result.loc[hydro_station, 'VAL_PPTS_ARIMA_Multiplicative'] = val_ppts_arima_multiplicative

    metrics_result.loc[hydro_station, 'VAL_LPTS_SWATPlus'] = val_lpts_swatplus
    metrics_result.loc[hydro_station, 'VAL_LPTS_ARIMA_Additive'] = val_lpts_arima_additive
    metrics_result.loc[hydro_station, 'VAL_LPTS_ARIMA_Multiplicative'] = val_lpts_arima_multiplicative
    
    

    
    
metrics_result.to_csv(f'../result/calval_metrics_result.csv')


Evaluate the forecast performance of different models


In [3]:
hydro_stations = ['Tangnaihai','Guide','Xunhua']
metrics_result = pd.DataFrame(
    index=hydro_stations,
    columns=['NNSE_PLTMSF','NNSE_ARIMA_Additive','NNSE_ARIMA_Multiplicative','NNSE_SWATPlus',
             'ZNRMSE_PLTMSF','ZNRMSE_ARIMA_Additive','ZNRMSE_ARIMA_Multiplicative','ZNRMSE_SWATPlus',
             'PPTS_PLTMSF','PPTS_ARIMA_Additive','PPTS_ARIMA_Multiplicative','PPTS_SWATPlus',
             'LPTS_PLTMSF','LPTS_ARIMA_Additive','LPTS_ARIMA_Multiplicative','LPTS_SWATPlus']
)
metrics_result.index.name = 'station'

for hydro_station in hydro_stations:
    
    add_arima_pred = pd.read_csv(
        f'../result/ARIMAPredData/seasonal_decompose_additive_arima_pred_{hydro_station.lower()}_2015_2019.csv',
        parse_dates=['date'],
        index_col=['date']
    )
    mul_arima_pred = pd.read_csv(
        f'../result/ARIMAPredData/seasonal_decompose_multiplicative_arima_pred_{hydro_station.lower()}_2015_2019.csv',
        parse_dates=['date'],
        index_col=['date']
    )
    pltmsf_pred = pd.read_csv(
        f'../result/SWATPlusPredUsingSimYearData/{hydro_station}_SWATPlus_corrected_pred_obs_2015_2019.csv',
        parse_dates=['date'],
        index_col=['date']
    )

    swatplus_pred = pd.read_csv(
        f'../result/SWATPlusPredUsingSimYearData/{hydro_station}_SWATPlus_pred_obs_2015_2019.csv',
        parse_dates=['date'],
        index_col=['date']
    )

    df_new = pd.DataFrame(pltmsf_pred.index)
    df_new.index.name = 'date'
    df_new['obs'] = pltmsf_pred['obs'].values
    df_new['pltmsf'] = pltmsf_pred['pred'].values
    df_new['add_arima'] = add_arima_pred['flow(m^3/s)'].values
    df_new['mul_arima'] = mul_arima_pred['flow(m^3/s)'].values
    df_new['swatplus'] = swatplus_pred['pred'].values

    nnse_pltmsf = NNSE(df_new['obs'].values, df_new['pltmsf'].values)
    nnse_add_arima = NNSE(df_new['obs'].values, df_new['add_arima'].values)
    nnse_mul_arima = NNSE(df_new['obs'].values, df_new['mul_arima'].values)
    nnse_swatplus = NNSE(df_new['obs'].values, df_new['swatplus'].values)

    znrmse_pltmsf = ZNRMSE(df_new['obs'].values, df_new['pltmsf'].values)
    znrmse_add_arima = ZNRMSE(df_new['obs'].values, df_new['add_arima'].values)
    znrmse_mul_arima = ZNRMSE(df_new['obs'].values, df_new['mul_arima'].values)
    znrmse_swatplus = ZNRMSE(df_new['obs'].values, df_new['swatplus'].values)

    ppts_pltmsf = PPTS(df_new['obs'].values, df_new['pltmsf'].values)
    ppts_add_arima = PPTS(df_new['obs'].values, df_new['add_arima'].values)
    ppts_mul_arima = PPTS(df_new['obs'].values, df_new['mul_arima'].values)
    ppts_swatplus = PPTS(df_new['obs'].values, df_new['swatplus'].values)

    lpts_pltmsf = LPTS(df_new['obs'].values, df_new['pltmsf'].values)
    lpts_add_arima = LPTS(df_new['obs'].values, df_new['add_arima'].values)
    lpts_mul_arima = LPTS(df_new['obs'].values, df_new['mul_arima'].values)
    lpts_swatplus = LPTS(df_new['obs'].values, df_new['swatplus'].values)

    metrics_result.loc[hydro_station, 'NNSE_PLTMSF'] = nnse_pltmsf
    metrics_result.loc[hydro_station, 'NNSE_ARIMA_Additive'] = nnse_add_arima
    metrics_result.loc[hydro_station, 'NNSE_ARIMA_Multiplicative'] = nnse_mul_arima
    metrics_result.loc[hydro_station, 'NNSE_SWATPlus'] = nnse_swatplus

    metrics_result.loc[hydro_station, 'ZNRMSE_PLTMSF'] = znrmse_pltmsf
    metrics_result.loc[hydro_station, 'ZNRMSE_ARIMA_Additive'] = znrmse_add_arima
    metrics_result.loc[hydro_station, 'ZNRMSE_ARIMA_Multiplicative'] = znrmse_mul_arima
    metrics_result.loc[hydro_station, 'ZNRMSE_SWATPlus'] = znrmse_swatplus
    
    metrics_result.loc[hydro_station, 'PPTS_PLTMSF'] = ppts_pltmsf
    metrics_result.loc[hydro_station, 'PPTS_ARIMA_Additive'] = ppts_add_arima
    metrics_result.loc[hydro_station, 'PPTS_ARIMA_Multiplicative'] = ppts_mul_arima
    metrics_result.loc[hydro_station, 'PPTS_SWATPlus'] = ppts_swatplus

    metrics_result.loc[hydro_station, 'LPTS_PLTMSF'] = lpts_pltmsf
    metrics_result.loc[hydro_station, 'LPTS_ARIMA_Additive'] = lpts_add_arima
    metrics_result.loc[hydro_station, 'LPTS_ARIMA_Multiplicative'] = lpts_mul_arima
    metrics_result.loc[hydro_station, 'LPTS_SWATPlus'] = lpts_swatplus

metrics_result.to_csv(f'../result/pred_metrics_result.csv')